In [2]:
!pip install h5py pyyaml

In [3]:
import os
import sys 
import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.14.0'

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()


11493376/11490434 [==============================] - 12s 1us/step


In [5]:
print(train_labels.shape, train_labels[2], sys.getsizeof(train_labels))
print(train_images.shape)
print(test_images.shape)
print(sys.getsizeof(train_images))

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

print(train_images.shape, train_images[0][200])

(60000,) 4 60096
(60000, 28, 28)
(10000, 28, 28)
47040128
(1000, 784) 0.0


In [18]:
train_images.shape
train_labels.shape

(1000,)

In [10]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation=tf.keras.activations.softmax)
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                 loss=tf.keras.losses.sparse_categorical_crossentropy,
                 metrics=['accuracy'])
    return model

model = create_model()
model.summary()

W0807 09:09:14.995683 140633344018240 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [22]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(checkpoint_dir)

#Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)
model = create_model()

model.fit(train_images, train_labels, epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])

training_1
Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 544/1000 [===============>..............] - ETA: 0s - loss: 1.5229 - acc: 0.5551
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 355us/sample - loss: 1.1571 - acc: 0.6730 - val_loss: 0.7366 - val_acc: 0.7710
Epoch 2/10
 608/1000 [=================>............] - ETA: 0s - loss: 0.4770 - acc: 0.8750
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 166us/sample - loss: 0.4236 - acc: 0.8790 - val_loss: 0.5410 - val_acc: 0.8260
Epoch 3/10
 608/1000 [=================>............] - ETA: 0s - loss: 0.2712 - acc: 0.9408
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 169us/sample - loss: 0.2855 - acc: 0.9300 - val_loss: 0.5332 - val_acc: 0.8260
Epoch 4/10
 544/1000 [===============>..............] - ETA: 0s - loss: 0.2314 - acc: 0.9412
Epoch 00004: saving model to training_1/cp.c

In [23]:
!ls {checkpoint_dir}

checkpoint		     cp.ckpt.data-00001-of-00002
cp.ckpt.data-00000-of-00002  cp.ckpt.index


In [24]:
model = create_model()

loss,acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy:{:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 77us/sample - loss: 2.3662 - acc: 0.1100
Untrained model, accuracy:11.00%


In [25]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy:{:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 43us/sample - loss: 0.4115 - acc: 0.8640
Restored model, accuracy:86.40%


In [34]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
         epochs=50, callbacks=[cp_callback],
         validation_data = (test_images, test_labels),
         verbose=0)

W0807 10:05:36.603660 140633344018240 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [35]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [36]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored 50ckpt model, accuracy:{:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 106us/sample - loss: 0.4729 - acc: 0.8800
Restored 50ckpt model, accuracy:88.00%
